In [1]:
import pandas as pd
import os
import os.path as op
import glob
from collections import Counter

In [2]:
tables = glob.glob("/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/PVOG/Allvogtables/*.txt")

In [3]:
exe = tables[0]

In [4]:
!head {exe}

#VOG3973: has 14 proteins sequences from 14 genomes
Siphoviridae	Enterobacteria phage DT571/2	KM979355	AJA41751.1	62099..62596	165	hypothetical protein
Siphoviridae	Enterobacteria phage DT57C	NC_027356	YP_009149879.1	61571..62068	165	hypothetical protein
Siphoviridae	Escherichia phage APCEc03	KR422353	AKO61505.1	68333..68830	165	hypothetical protein
Siphoviridae	Escherichia phage Akfv33	NC_017969	YP_006382424.1	71768..72265	165	hypothetical protein
Siphoviridae	Escherichia phage Eps7	NC_010583	YP_001837059.1	76012..76494	160	hypothetical protein
Siphoviridae	Escherichia phage T5	NC_005859	YP_006951.1	73657..74154	165	hypothetical protein
Siphoviridae	Escherichia phage vB_EcoS_FFH1	NC_024139	YP_009031742.1	71772..72269	165	hypothetical protein
Siphoviridae	Pectobacterium phage My1	NC_018837	YP_006906369.1	72254..72754	166	hypothetical protein
Siphoviridae	Salmonella phage Shivani	NC_028754	YP_009194757.1	70669..71166	165	hypothetical protein


In [14]:
names = ['family','species','genome_id','protein_id','start...stop','prot_len','function']

In [17]:
dflist = []
for exe in tables:
    with open(exe) as ih:
        dlist = []
        for l in ih:
            if l.startswith("#"):
                vog = l.split(":")[0].replace("#","")
            else:
                dlist.append(dict(zip(names, l.strip().split("\t"))))
        df = pd.DataFrame(dlist)
        df['vog'] = vog
        dflist.append(df)

In [32]:
all_vogs = pd.concat(dflist)
all_vogs.to_csv("/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/PVOG/all_proteins_tbl.csv", index=False)

In [44]:
with open(exe) as ih:
    dlist = []
    for l in ih:
        if l.startswith("#"):
            vog = l.split(":")[0].replace("#","")
        else:
            toks= dict(zip(names, l.strip().split("\t")))
            dlist.append(toks)
    functions = list(set([i['function'] for i in dlist if (i['function'] != 'hyphothetical protein') & (i['function'] != 'hypothetical protein')]))
    
    if len(functions) == 0:
        func_string = "hypothetical protein"
    else:
        func_string = ",".join(functions)
        
    famcount = Counter([i['family'] for i in dlist])
    fam_string = ";".join(["{i}({count})".format(i=i, count = famcount[i]) for i in famcount])
    dct = {'vog':vog, 'family':fam_string, 'function':func_string}

In [45]:
func_string

'hypothetical protein'

In [63]:
dctlist = []

for exe in tables:
    with open(exe) as ih:
        dlist = []
        for l in ih:
            if l.startswith("#"):
                vog = l.split(":")[0].replace("#","")
            else:
                toks= dict(zip(names, l.strip().split("\t")))
                dlist.append(toks)
        try:
            functions = list(set([i.get('function','-') for i in dlist if (i.get('function','-') != 'hyphothetical protein') & (i.get('function','-') != 'hypothetical protein')]))
        except Exception as inst:
            print(dlist)
            print(inst)
            break

        if len(functions) == 0:
            func_string = "hypothetical protein"
        else:
            func_string = ",".join(functions)

        famcount = Counter([i['family'] for i in dlist]).most_common()
        fam_string = ";".join(["{i}({count})".format(i=famcount[i][0], count = famcount[i][1]) for i in range(0, len(famcount))])
        dct = {'vog':vog, 'family':fam_string, 'function':func_string, 'members':len(dlist)}
        dctlist.append(dct)

In [64]:
vdf = pd.DataFrame(dctlist)
vdf.head()

,family,function,members,vog
0,Siphoviridae(14),hypothetical protein,14,VOG3973
1,Myoviridae(6);NA(4);Siphoviridae(2),hypothetical protein,12,VOG5636
2,Siphoviridae(20),"gp74,gp76,gp75,gp73,gp72,gp68",20,VOG2494
3,Podoviridae(6);NA(5),hypothetical protein,11,VOG5730
4,Myoviridae(5),putative nucleotidyltransferase,5,VOG4157


In [66]:
vdf.sort_values(by = 'members', ascending=False).head(20)

,family,function,members,vog
1187,Siphoviridae(4757);Myoviridae(1960);Podovirida...,"CB,gp355,putative phosphoribosyl-ATPpyrophosph...",8131,VOG4543
6953,Siphoviridae(1297);Myoviridae(502);Podoviridae...,"may contain ATP/GTP binding motif,Terminase,te...",2299,VOG4544
3147,Siphoviridae(1336);Myoviridae(298);NA(106);Pod...,"lysin,tail morphogenetic protein, tape masure ...",1762,VOG4545
8474,Myoviridae(782);Siphoviridae(538);Podoviridae(...,"Primase/Helicase,T7-like primase/helicase,reco...",1644,VOG0025
8910,Myoviridae(603);Siphoviridae(431);Podoviridae(...,"phage tail collar domain-containing protein,st...",1289,VOG4546
5146,Siphoviridae(798);Myoviridae(233);Podoviridae(...,"lysin,bacteriophage SPbeta N-acetylmuramoyl-L-...",1259,VOG4547
1369,Myoviridae(508);Siphoviridae(440);Podoviridae(...,"gp41, helicase,ATP-dependent helicase,SNF2 dom...",1078,VOG4548
4696,Siphoviridae(444);Podoviridae(294);Myoviridae(...,"N4 gp39-like protein,putative DNA polymerase 2...",997,VOG0026
7136,Myoviridae(536);Siphoviridae(266);NA(35);Podov...,"glutaredoxin 3,glutaredoxin protein,Phi92_gp06...",869,VOG4549
2266,Myoviridae(697);Siphoviridae(88);NA(42);Podovi...,"AB1gp80,gp38, phage baseplate assembly protein...",832,VOG4550


In [67]:
vdf[['vog','family','function','members']].to_csv("/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/PVOG/allvog_id_summary.csv",index=False)

In [21]:
fc = all_vogs.groupby(['vog','function'])['family'].count().reset_index().rename(columns = {'family':'function_count'})

In [31]:
fc[(fc['function'] != 'hypothetical protein') & (fc['function'] != 'putative predicted product') & (fc['function'] != 'conserved hypothetical protein')].sort_values(by=['vog', 'function_count'], ascending=False).drop_duplicates(subset='vog', keep='first')

fc.sort_values(by=['vog','function_count'], ascending = False)

,vog,function,function_count
43676,VOG9999,hypothetical protein,3
43675,VOG9998,hypothetical protein,3
43674,VOG9997,hypothetical protein,3
43673,VOG9996,hypothetical protein,3
43670,VOG9995,putative variable tail fiber protein,1
43671,VOG9995,tail fiber,1
43672,VOG9995,tail fiber protein,1
43668,VOG9994,hypothetical protein,2
43669,VOG9994,putative predicted product,1
43667,VOG9993,hypothetical protein,2


In [ ]:
for v in all_vo